In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
# tfds.disable_progress_bar()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 示例1

In [2]:
embedding_layer = layers.Embedding(1000, 5)
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.02617301,  0.02699072, -0.00274373, -0.00347971, -0.00272981],
       [-0.01190905,  0.02377695,  0.0427632 ,  0.00268843, -0.02440553],
       [ 0.02156243,  0.02496378, -0.01283579, -0.01258569,  0.0194456 ]],
      dtype=float32)

In [3]:
result.shape

TensorShape([3, 5])

## 示例2

In [4]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.numpy()

array([[[-0.00457449, -0.0446418 ,  0.006168  , -0.01145371,
         -0.01278301],
        [-0.02617301,  0.02699072, -0.00274373, -0.00347971,
         -0.00272981],
        [-0.01190905,  0.02377695,  0.0427632 ,  0.00268843,
         -0.02440553]],

       [[ 0.02156243,  0.02496378, -0.01283579, -0.01258569,
          0.0194456 ],
        [ 0.00010318,  0.02293346, -0.01302607, -0.02196333,
          0.00073823],
        [ 0.00598999,  0.04244087, -0.02477986, -0.00411811,
          0.00882529]]], dtype=float32)

In [5]:
result.shape

TensorShape([2, 3, 5])

# 基于 IMDB 电影品论来训练情感分类器

In [3]:
(train_data, test_data), info = tfds.load(
    "imdb_reviews/subwords8k", 
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    with_info = True, 
    as_supervised = True,
)

In [8]:
encoder = info.features["text"].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

In [9]:
# 电影评论的长度可以不同，使用 padded_batch 标准化评论的长度
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()
train_labels.numpy()

array([[ 693, 1922,   83, ...,    0,    0,    0],
       [ 768,  109, 7968, ...,    0,    0,    0],
       [  19, 7301,    6, ...,    0,    0,    0],
       ...,
       [  19,  651, 3966, ...,   23, 3850, 7975],
       [  62,   27,    9, ...,    0,    0,    0],
       [  12,   80,    7, ...,    0,    0,    0]])

array([0, 0, 1, 0, 0, 0, 1, 1, 0, 0])

In [10]:
embedding_dim = 16
model = keras.Sequential([
    layers.Embedding(encoder.vocab_size, embedding_dim),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation = "relu"),
    layers.Dense(1),
])
model.summary()
model.compile(
    optimizer = "adam", 
    loss = tf.keras.losses.BinaryCrossentropy(from_logits = True), 
    metrics = ["accuracy"],
)
history = model.fit(
    train_batches,
    epochs = 10,
    validation_data = test_batches,
    validation_steps = 20,
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2500/2500 [==============================] - 4s 1ms/step - loss: 0.6073 - accuracy: 0.5907 - val_loss: 0.4243 - val_accuracy: 0.7400
Epoch 2/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.2954 - accuracy: 0.8770 - 

# 检索学习的嵌入向量

In [11]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(8185, 16)


In [16]:
import io

encoder = info.features["text"].encoder
out_v = io.open("vecs.tsv", "w", encoding = "utf-8")
out_m = io.open("meta.tsv", "w", encoding = "utf-8")
for num, word in enumerate(encoder.subwords):
    vec = weights[num + 1]
    out_m.write(word + "\n")
    out_v.write("\t".join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

5

190

3

195

3

...

170

4

175

5

178